In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    "C:\\Users\\USER\\OneDrive\\Desktop\\dataset",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    "C:\\Users\\USER\\OneDrive\\Desktop\\dataset",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 3074 images belonging to 9 classes.
Found 763 images belonging to 9 classes.


In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of ResNet-50 except the top layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)


Epoch 1/10
96/96 [==============================] - 181s 2s/step - loss: 0.6518 - accuracy: 0.8225 - val_loss: 0.4259 - val_accuracy: 0.8655
Epoch 2/10
96/96 [==============================] - 162s 2s/step - loss: 0.1637 - accuracy: 0.9464 - val_loss: 0.2930 - val_accuracy: 0.8927
Epoch 3/10
96/96 [==============================] - 167s 2s/step - loss: 0.1459 - accuracy: 0.9513 - val_loss: 0.2862 - val_accuracy: 0.9022
Epoch 4/10
96/96 [==============================] - 169s 2s/step - loss: 0.1307 - accuracy: 0.9550 - val_loss: 0.2305 - val_accuracy: 0.9266
Epoch 5/10
96/96 [==============================] - 161s 2s/step - loss: 0.0893 - accuracy: 0.9688 - val_loss: 0.3076 - val_accuracy: 0.9062
Epoch 6/10
96/96 [==============================] - 155s 2s/step - loss: 0.0850 - accuracy: 0.9724 - val_loss: 0.2716 - val_accuracy: 0.9212
Epoch 7/10
96/96 [==============================] - 150s 2s/step - loss: 0.0789 - accuracy: 0.9740 - val_loss: 0.2036 - val_accuracy: 0.9361
Epoch 8/10
96

In [9]:
test_generator = train_datagen.flow_from_directory(
    "C:\\Users\\USER\\OneDrive\\Desktop\\dataset",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  
)

evaluation = model.evaluate(test_generator)
print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}")


Found 763 images belonging to 9 classes.
24/24 [==============================] - 33s 1s/step - loss: 0.2767 - accuracy: 0.9069
Test Loss: 0.2767234444618225, Test Accuracy: 0.9069462418556213


In [10]:
model.save('vegetable_leaf_classifier.h5')


c:\Users\USER\OneDrive\Desktop\dataset\env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
